## Preprocessing

In [87]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [88]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df_low = application_df.drop(columns=['EIN', 'NAME'])
application_df_low

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [89]:
# Determine the number of unique values in each column.
application_df_nu = application_df_low.nunique()
application_df_nu

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [90]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_df_low['APPLICATION_TYPE'].value_counts()


APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [91]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = ["T2", "T12", "T9", "T13", "T14", "T15", "T17", "T25", "T29", "T10"]

# Replace in dataframe
for app in application_types_to_replace:
    application_df_low['APPLICATION_TYPE'] = application_df_low['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df_low['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
Other      804
T8         737
T7         725
Name: count, dtype: int64

In [92]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
nu_counts_class = application_df_low['CLASSIFICATION'].value_counts()
nu_counts_class

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C1248        1
C6100        1
C1820        1
C1900        1
C2150        1
Name: count, Length: 71, dtype: int64

In [93]:
# You may find it helpful to look at CLASSIFICATION value counts > 1
for CLASSIFICATION, count in nu_counts_class.items():
        if count > 1:
                print(f"{CLASSIFICATION}: {count}")

C1000: 17326
C2000: 6074
C1200: 4837
C3000: 1918
C2100: 1883
C7000: 777
C1700: 287
C4000: 194
C5000: 116
C1270: 114
C2700: 104
C2800: 95
C7100: 75
C1300: 58
C1280: 50
C1230: 36
C1400: 34
C7200: 32
C2300: 32
C1240: 30
C8000: 20
C7120: 18
C1500: 16
C6000: 15
C1800: 15
C1250: 14
C8200: 11
C1238: 10
C1278: 10
C1237: 9
C1235: 9
C7210: 7
C1720: 6
C2400: 6
C4100: 6
C1257: 5
C1600: 5
C1260: 3
C2710: 3
C0: 3
C3200: 2
C1256: 2
C1246: 2
C1234: 2
C1267: 2


In [94]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
nu_counts_class_series = pd.Series(nu_counts_class)
classifications_to_replace = nu_counts_class_series[nu_counts_class_series < 1000].index

# Replace in dataframe
for cls in classifications_to_replace:
     application_df_low['CLASSIFICATION'] = application_df_low['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df_low['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [95]:
application_df_low.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34299 entries, 0 to 34298
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   APPLICATION_TYPE        34299 non-null  object
 1   AFFILIATION             34299 non-null  object
 2   CLASSIFICATION          34299 non-null  object
 3   USE_CASE                34299 non-null  object
 4   ORGANIZATION            34299 non-null  object
 5   STATUS                  34299 non-null  int64 
 6   INCOME_AMT              34299 non-null  object
 7   SPECIAL_CONSIDERATIONS  34299 non-null  object
 8   ASK_AMT                 34299 non-null  int64 
 9   IS_SUCCESSFUL           34299 non-null  int64 
dtypes: int64(3), object(7)
memory usage: 2.6+ MB


In [96]:
# Convert categorical data to numeric with `pd.get_dummies`
df_dummies = pd.get_dummies(
    application_df_low,
    columns=[
        "APPLICATION_TYPE",
        "AFFILIATION", "CLASSIFICATION",
        "USE_CASE", "ORGANIZATION", "INCOME_AMT",
        "SPECIAL_CONSIDERATIONS"
    ]
)
print(df_dummies)

       STATUS   ASK_AMT  IS_SUCCESSFUL  APPLICATION_TYPE_Other  \
0           1      5000              1                    True   
1           1    108590              1                   False   
2           1      5000              0                   False   
3           1      6692              1                   False   
4           1    142590              1                   False   
...       ...       ...            ...                     ...   
34294       1      5000              0                   False   
34295       1      5000              0                   False   
34296       1      5000              0                   False   
34297       1      5000              1                   False   
34298       1  36500179              0                   False   

       APPLICATION_TYPE_T19  APPLICATION_TYPE_T3  APPLICATION_TYPE_T4  \
0                     False                False                False   
1                     False                 True             

In [97]:
application_df_low.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34299 entries, 0 to 34298
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   APPLICATION_TYPE        34299 non-null  object
 1   AFFILIATION             34299 non-null  object
 2   CLASSIFICATION          34299 non-null  object
 3   USE_CASE                34299 non-null  object
 4   ORGANIZATION            34299 non-null  object
 5   STATUS                  34299 non-null  int64 
 6   INCOME_AMT              34299 non-null  object
 7   SPECIAL_CONSIDERATIONS  34299 non-null  object
 8   ASK_AMT                 34299 non-null  int64 
 9   IS_SUCCESSFUL           34299 non-null  int64 
dtypes: int64(3), object(7)
memory usage: 2.6+ MB


In [98]:
# Split our preprocessed data into our features and target arrays
y = df_dummies["IS_SUCCESSFUL"]
X = df_dummies.drop(columns=["IS_SUCCESSFUL"])

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=40)

In [99]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [100]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()

# Input layer
nn.add(tf.keras.layers.Dense(units=200, activation="relu", input_dim=X.shape[1], name="input"))
nn.add(tf.keras.layers.Dropout(0.3))

# First hidden layer
nn.add(tf.keras.layers.Dense(units=150, activation="relu", name="hidden_1"))
nn.add(tf.keras.layers.Dropout(0.2))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=100, activation="relu", name="hidden_2"))
nn.add(tf.keras.layers.Dropout(0.2))

# # Third hidden layer
nn.add(tf.keras.layers.Dense(units=50, activation="relu", name="hidden_3"))
nn.add(tf.keras.layers.Dropout(0.2))

# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=20, activation="relu", name="hidden_4"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid", name="output"))

# Check the structure of the model
nn.summary()

c:\Users\Sibal_1w46aws\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input (Dense)                   │ (None, 200)            │         8,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 200)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ hidden_1 (Dense)                │ (None, 150)            │        30,150 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 150)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ hidden_2 (Dense)                │ (None, 100)            │        15,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ hidden_3 (Dense)                │ (None, 50)             │         5,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ hidden_4 (Dense)                │ (None, 20)             │         1,020 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 1)              │            21 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 59,941 (234.14 KB)

 Trainable params: 59,941 (234.14 KB)

 Non-trainable params: 0 (0.00 B)

In [101]:
# class AccuracyThresholdCallback(tf.keras.callbacks.Callback):
#     def __init__(self, threshold, save_path):
#         super().__init__()
#         self.threshold = threshold
#         self.save_path = save_path

#     def on_epoch_end(self, epoch, logs=None):
#         accuracy = logs.get("accuracy")
#         if accuracy and accuracy > self.threshold:
#             print(f"\nValidation accuracy {accuracy:.4f} exceeded threshold {self.threshold:.2f}. Saving model...")
#             self.model.save(f"{self.save_path}_{accuracy:.4f}.h5")

# # Use the custom callback
# threshold_callback = AccuracyThresholdCallback(threshold=0.75, save_path="models/best_model")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='models/best_model.keras',
    monitor='accuracy',
    mode='max',
    save_best_only=True,
    save_weights_only=False,
    verbose=1,
    save_freq='epoch'
)

In [102]:
# Compile the model
nn.compile(
    loss="Huber",#"binary_crossentropy",
    optimizer=tf.keras.optimizers.Nadam(learning_rate=0.0001),#"adam",
    metrics=["accuracy"]
)

In [ ]:
# Train the model
fit_model = nn.fit(
  X_train_scaled,
  y_train,
  epochs=2000,
  batch_size=500,
  callbacks=[checkpoint_callback]
)

Epoch 1/2000
47/49 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5296 - loss: 0.1260
Epoch 1: accuracy improved from -inf to 0.55592, saving model to models/best_model.keras
49/49 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.5312 - loss: 0.1259
Epoch 2/2000
46/49 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6263 - loss: 0.1179
Epoch 2: accuracy improved from 0.55592 to 0.64184, saving model to models/best_model.keras
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.6276 - loss: 0.1178
Epoch 3/2000
46/49 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.6768 - loss: 0.1110
Epoch 3: accuracy improved from 0.64184 to 0.68141, saving model to models/best_model.keras
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.6772 - loss: 0.1109
Epoch 4/2000
47/49 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7024 - loss: 0.1050
Epoch 4: accuracy improved from 0.68141 to 0.69932, saving model to models/best_model.keras
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.7022 - loss: 0.

In [ ]:
# Evaluate the model using the test data
best_model = tf.keras.models.load_model("models/best_model.keras")
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

322/322 - 1s - 3ms/step - accuracy: 0.7311 - loss: 0.0934
Loss: 0.093438059091568, Accuracy: 0.7310981750488281


In [ ]:
# Export our model to HDF5 file
best_model.save("models/best_model.h5", save_format='h5')

In [ ]:
tf.keras.utils.plot_model(best_model, to_file='model_structure.png', show_shapes=True)

You must install pydot (`pip install pydot`) for `plot_model` to work.
